# AIFFEL 대전 1기 Going Deeper CV Lecture4
## Date: APRIL 15, 2021
## 물체를 분리하자! - 세그멘테이션 살펴보기

# 1. 세그멘테이션 문제의 종류
1) 시맨틱 세그멘테이션(semantic segmentation)  
2) 인스턴스 세그멘테이션(instance segmentatio)

![세그멘테이션종류](https://aiffelstaticprd.blob.core.windows.net/media/original_images/semantic_vs_instance.png)  

1. 위 그림 중 좌측 상단그림은 이미지 내에서 양의 위치(localization)와 양의 존재를 판별(classificatio)하고 있다.  
2. 우측 상단그림은 개별 개체에 대해 식별하고 있는 객체 인식(object detection)을 보여준다.  
3. 좌측 하단그림은 시맨틱 세그멘테이션이며 이미지 속 객체를 구분하고 있다.  
4. 우측 하단그림은 인스턴스 세그멘테이션이며 객체별 개체별 구분하고 있다.  

**즉, 인스턴스 세그멘테이션은 개체별 구분이 가능한 세그멘테이션 기법이다.**

## 1) 시맨틱 세그멘테이션(Semantic Segmentation)
***
![1](https://aiffelstaticprd.blob.core.windows.net/media/images/segnet_demo.max-800x600.png)  
[SegNet 데모 사이트](https://mi.eng.cam.ac.uk/projects/segnet/#demo)  

위 사이트를 통해 시맨틱 세그멘테이션은 12개의 라벨로 구분하고 있음을 확인.  

![2](https://aiffelstaticprd.blob.core.windows.net/media/images/u-net.max-800x600.png)  

1. 위 모델 구조는 시맨틱 세그멘테이션의 대표적인 모델인 U-Net의 구조  
2. 입력: 572x572  
3. 출력: 388x388 크기 + 두 가지의 클래스를 가진 세그멘테이션 맵이 나온다.  
4. 두 가지 클래스 예시: 인물모드(사람과 배경), 의료 영상(정상세포와 암 세포) 등  
5. 이전에 봤던 이미지 분류나 물체 검출 문제보다 확실히 큰 출력값: 세그멘테이션을 위해서 이미지의 각 픽셀에 해당하는 영역의 클래스별 정보가 필요하기 때문.  



## 2) 인스턴스 세그멘테이션(Instance Segmentation)
***
1. 같은 클래스 내에서도 각 개체(instance)들을 분리하해 세그멘테이션 수행.  
2. 아이디어: 물체 검출(object detection) 모델로 각 개체를 구분하고 이후에, 각 개체 별로 시맨틱 세그멘테이션 수행.(Mask R-CNN이 적합)  
3. Mask R-CNN: Faster-R-CNN의 ROI(Regio-of-Interest) Pooling Layer(ROIPool)  
4. 위 개념을 개선해 정확한 세그멘테이션을 위한 **ROIAlign**, **클래스별 마스크 분리**라는 아이디어를 통해 클래스별 Object Detection과 시멘틱 세그멘테이션을 하나의 Task로 엮어냄.  

![RoI](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-5-L-RoIPool-01.max-800x600.png)  

512x512 이미지를 VGG16을 거치면 16x16x512로 변환되며 scale factor은 32가 된다. 만약 가로 200, 세로 145 픽셀짜리 RoI영역을 16x16으로 분할된 영역에 끼워맞추려고 하면 가로 6.25, 세로 4.53이 되서 RoI영역 밖이 포함되기도 하고 자투리 영역이 버려지는 경우도 생긴다.  

또한 3x3 RoIPool을 하게 되면 아래와 같이 가로 6, 세로4의 영역에서 적용되어 맨 아래 가로 한 줄은 버려지게되는 좋지 않은 결과가 생성된다.  
![pool](https://miro.medium.com/max/700/0*H9dzv-y3dmmDdHIt.png)  

***

# Understanding Region of Interest — (RoI Align and RoI Warp)
*아래 글은 [여기](https://towardsdatascience.com/understanding-region-of-interest-part-2-roi-align-and-roi-warp-f795196fc193)를 해석한 글입니다.*  

## RoI 풀링을 왜 수정해야 하나?
이전 글(위의 글)에서 기억하시겠지만, RoI Pooling은 한 가지 큰 문제가 있다. 이 과정에서 많은 데이터가 손실된다.  

![RoI pooling](https://miro.medium.com/max/700/0*BsrnKGN-HZjMWRHi.png)  

그럴 때마다 객체에 대한 정보가 손실된다. 이는 곧 전체 모델의 정밀도(precision)을 낮췄는데 똑똑한 사람들이 이게 대해 생각했다.  

# Setup
시작하기 전에 우리의 모델에 대한 빠른 설명을 하겠다.  

![Original Mask R-CNN architecture](https://miro.medium.com/max/700/0*ZhruhfkX0fu8Nf1V.png)  

![Mask R-CNN output](https://miro.medium.com/max/700/0*FSAVHzp_ejTq-AKX.png)  

우리는 테스트를 위해 Mask R-CNN 네트워크를 사용할 것이다. 이 시스템을 사용하는 유일한 이유는 이러한 종류의 네트워크가 정확한 풀링 계층을 통해 더 많은 이점을 얻을 수 있기 때문에 RoI Align과 RoI Pooling 간의 차이를 더 쉽게 보여줄 수 있기 때문입니다. RoI Pooling이 되기 전까지는 어떤 네트워크를 사용하는지는 중요하지 않습니다. 따라서 설정은 그대로 유지되며 다음과 같습니다.  

![Model featur mapping process](https://miro.medium.com/max/700/0*JArsNcfm3TyRj1Q8.png)  

우리의 모델은 **512x512x(width x height x RGB)** 크기의 입력 이미지를 받고 VGG16을 통해 **16x16x512** 특성 맵으로 매핑된다. scale factor은 32이다.  

다음으로 우리는 제안된 RoI(**145x200** box)를 사용해서 특성 맵에 매핑하려고 한다. 객체의 차원 모두가 32로 나눠지지 않기 때문에, RoI를 그리드에 맞추고 있지 못하고 있다.  

![RoI placement](https://miro.medium.com/max/700/0*JXAuro-N0Nl4zDt_.png)  

- **(9.25, 6)** - 좌측 상단 코너  
- **6.25** - 너비  
- **4.53** - 높이  

다시 한 번 풀링 레이어를 3x3 크기로 선택해 최종 결과 모양이 3x3x512가 되도록 한다. (이미지에 쉽게 표시할 수 있도록 하기 위한 임의의 예시일 뿐. 풀링 계층의 크기는 다를 수 있다.)  

![Pooling layer](https://miro.medium.com/max/700/0*6YgoeWBShTWx25bD.png)  

여기까지는 RoI Pooling와 똑같아 보인다.  


## RoI Align 소개
RoI 풀링과 RoI 정렬의 주요 차이점은 양자화(quantization)입니다. **RoI Align은 데이터 풀링에 양자화(quantization)를 사용하지 않습니다.** Fast R-CNN이 양자화(quantization)를 두 번 적용하는 것을 알고 있습니다. 매핑 프로세스에서는 처음이고 풀링 프로세스에서는 두 번째입니다.  

![Quantization when mapping and pooling](https://miro.medium.com/max/700/0*H9dzv-y3dmmDdHIt.png)  

우리는 원래의 RoI를 9개의 동일한 크기의 박스로 나누고 그 안에 이중 보간법(bilinear interpolation)을 적용함으로써 그것(데이터 풀링에 양자화)을 skip할 수 있다. 그러면 상자를 정의하겠습니다.  

![RoI Box size](https://miro.medium.com/max/700/0*RNy9uPsv5acoTtPg.png)  

각 상자 크기는 매핑된 RoI의 크기와 풀링 계층의 크기에 따라 결정됩니다. 우리는 **3x3** 풀링 레이어를 사용하기 때문에 매핑된 RoI(**6.25x4.53**)를 3으로 분할해야 합니다. 그러면 높이가 **1.51**이고 폭이 **2.08**인 상자를 얻을 수 있습니다(더 쉽게 만들기 위해 값을 반올림합니다). 이제 상자를 매핑된 RoI에 넣을 수 있습니다.  

![RoI divided into boxes](https://miro.medium.com/max/700/0*CC8CdUxkIm_DVT5V.png)  

첫 번째 상자(왼쪽 위)를 보면 여섯 개의 다른 그리드 셀을 포괄하는 것을 알 수 있습니다. 풀링 계층에 대한 값을 추출하려면 해당 계층에서 일부 데이터를 샘플링해야 합니다. 데이터를 샘플링하려면 상자 안에 **4개의 샘플링 포인트**를 생성해야 합니다.  

![Sampling points distribution](https://miro.medium.com/max/700/0*R_Nbks0OUyRZPRH3.png)  

**상자의 높이와 너비를 3으로 나누어** 각 점이 어디에 위치해야 하는지 계산할 수 있습니다.  

이 경우 첫 번째 점(왼쪽 위) 좌표를 다음과 같이 계산합니다.  

- X = X_box + (width/3) \* 1 = 9.94  
- Y = Y_box + (height/3) \* 1 = 6.50  

두 번째 포인트(왼쪽 아래) 좌표를 다음과 같이 계산합니다.(Y만 바꾸면 됩니다.)  
- X = X_box + (width/3) \* 1 = 9.94  
- Y = Y_box + (height/3) \*2 = 7.01  

이제 모든 점이 확보되면 이 상자의 샘플 데이터에 이선형 보간법을 적용할 수 있습니다. 이선형 보간은 일반적으로 이미지 처리에서 색상을 샘플링하는 데 사용되며 방정식은 다음과 같습니다.  

![Bilinear Interploation equation](https://miro.medium.com/max/700/1*J7VPE_DrDTwSluzN_8RJKQ.png)  

위 수식을 이해하는 대시 아래 그림을 보고 이해하자.  

![Bilinear Interpolation for the first point](https://miro.medium.com/max/700/0*EEREnVrhY4U83vUa.png)  

첫 번째 점을 상자로부터 얻을 때 당신은 그것과 가장 가까운 이웃한 점(아직 그 점을 take하지 않았다면)과 연결해야 한다. (정확히 가운데와 연결될 것이다.) 이 경우에는 우리의 좌표는 **(9.44, 6.50)** 이다. 최상단 왼쪽의 가장 가까운 중앙 셀의 방향은 **(9.50,6.50)** 이다. 만약 우리의 점이 그리드보다 0.01만 높으면 좌표는 (9.50, 5.50)이 될 것이다.)  
그 다음 우리는 좌측 하단 점을 선택해야 하고 가장 가까운 점은 **(9.50, 7,50)** 이 된다. 같은 방법으로 **(10.50, 6.50)** 과 **(10.50, 7.50)** 을 선택할 수 있다.(우측 상단과 우측 하단의 점으로). RoI 위에서는 첫 번째 점(0.14)에 대한 값을 얻기 위한 전체 계산을 볼 수 있습니다.  

![Bilinear Interpolation for the second point](https://miro.medium.com/max/700/0*pNXjoftdbR9W4tNW.png)  
이번엔 아래와 같이 보간법을 적용하면:  

- 좌측 상단: (10.50, 6.50)  
- 좌측 하단: (10.50, 7.50)  
- 우측 상단: (11.50, 6.50)  
- 우측 하단: (11.50, 7.50)  

당신은 패턴이 보이기 시작할 것이다. 여기 다른 포인트에 대한 그림이 있다.  
![Bilinear Interpolation for the third point](https://miro.medium.com/max/700/0*czN31BxYWrujn-qD.png)  
- 좌측 상단: (9.50, 6.50)  
- 좌측 하단: (9.50, 7.50)  
- 우측 상단: (10.50, 6.50)  
- 우측 하단: (10.50, 7.50)  

![Bilinear Interpolation for the fourth poing](https://miro.medium.com/max/700/0*J78DE2M7QdwNns_T.png)  
- 좌측 상단: (10.50, 6.50)  
- 좌측 하단: (10.50, 7.50)  
- 우측 상단: (11.50, 6.50)  
- 우측 하단: (11.50, 7.50)  

이제 우리는 모든 점을 계산했고 **Max Pooling**을 적용할 수 있다. (만약 Avg pooling을 하고 싶다면 가능하다.)  
![First Box Pooling](https://miro.medium.com/max/700/0*h4cpIjHPjRx2dMRc.png)  

여기서 모든 보간법 실행을 보여주지 않을 것이다. 왜냐하면 이미 당신은 방법을 터득했고 그것을 다 적기엔 너무 많기 때문이다. 여기서 나는 RoIAlign을 이용해 어떻게 RoI으로부터 데이터를 pooling하는지에 관한 전체 과정을 보여줄 것이다.  
![RoIAlign pooling process(open image in new tab to check all calculations](https://miro.medium.com/max/700/0*7WFmQBxoOCPu2BDJ.gif)  

그리고 물론, 이 과정은 모든 레이어에 적용되므로 최종 결과에 512개의 레이어가 포함된다. (feature map 입력과 동일)  
![RoIAlign full size](https://miro.medium.com/max/445/0*KGrcXozWbZOfDfqk.png)  

feature map 모든 셀 내부에 우리의 샘플링 포인트를 배치하지 않더라도 이중 보간법으로 데이터를 추출.  
이 경우 11x6, 11x7, 11x8, 11x9, 11x10, 13x6, 13x7, 13x8, 13x9, 13x6, 15x6, 15x7, 15x8, 15x10 셀에는 포인트가 없습니다. 두 번째 점 계산(첫 번째 상자)을 살펴보면 점이 셀 10x6에 있더라도 셀 11x6 및 11x7을 이선형 보간법으로 사용합니다.  
RoIAlign과 RoIPoling의 데이터 손실/이득을 비교하면 RoIAlign이 전체 영역에 대해 pooling하는 것을 확인할 수 있다.  

![Comparing RoIAlign(left) and RoIPooling(right) data sources.](https://miro.medium.com/max/700/0*EUJOm7bE0zM6FxFE.png)  

- 초록색: 풀링에 사용되는 추가 데이터를 의미.  
- 파란색(양쪽 음영 모두): 풀링 시 손실된 데이터를 의미.  

## RoIWarp - meet me in the middle(중간에서 만나자)
다중 작업(multip-task) 네트워크 케스케이드(작은 폭포)를 통해 인스턴스 인식(instance-aware) 시멘틱 세그멘테이션에 도입된 세 번째 데이터 풀링 방법이 있는데 이를 RoIWarp라고 한다. RoIWarp의 아이디어는 RoIAlign과 거의 같으며, 유일한 차이점은 RoIWarp가 feature map에 대한 RoI 매핑을 정량화(quantizing)한다는 것이다.  
- [Instance-aware semantic segmentation via multi-task network cascades](https://arxiv.org/pdf/1512.04412.pdf)  

![RoI Warp](https://miro.medium.com/max/700/0*Yzsw9XOLtRjK-qMw.png)  

그리고 만약 당신이 데이터 손실/이득을 본다면:  
![RoI Wapr data lost/data gain](https://miro.medium.com/max/700/0*sDjOScbSjgoUAY6z.png)  

우리는 오직 bilinear interpolation(이선 보간법)에 의해 소량만 손실되고 있다.  

## RoIWarp와 RoIAlign이 어떻게 정밀도에 영향을 미치는가?
우리가 Mask R-CNN 논문을 봤을 때 우리는 논의할 중요한 몇 가지가 있었다. 첫 번째는 ResNet-50-C4(with stride 16)에 서로 다른 RoI 계층을 적용할 때의 평균 정밀도의 변화이다.  
![table1](https://miro.medium.com/max/640/1*LzXWxztqOjhW3219eWCAlg.png)  

RoIwarp를 적용했을 때 약간의 개선이 있을 뿐이지만 RoIAlign을 적용하면 정밀도가 크게 향상된다. stride을 높이면 상승폭이 커집니다.  
![table2](https://miro.medium.com/max/641/1*mKEow8X4uffLqF-TSbA1NA.png)  

여기서 APbb는 경계 박스를 탐지하기 위한 평균 정밀도이다. 테스트는 ResNet-50-C5(with stride 32)에서 수행된다.  

## 요약
R-CNN과 같은 모델 정확도를 향상시키려면 RoI 풀링을 이해하는 것이 중요하다. 2014년 Fast R-CNN에 관한 논문에서 제안된 표준 접근법과 2018년 Mask R-CNN에 관한 논문에서 제안된 새로운 접근법 사이에는 상당한 차이가 있다. 이는 이러한 방법들이 특정 네트워크에만 적용된다는 의미가 아니다. 우리는 Fast R-CNN에서 RoIAlign, Mask R-CNN에서 RoIPooling을 쉽게 사용할 수 있지만 당신은 RoIAlign이 평균적으로 더 나은 정밀도를 제공한다는 것을 기억해야 한다.  

저는 RoI Pooling에 대한 많은 글을 계산하지 않고 봤기 때문에 제 설명이 이해하기 쉬웠으면 합니다. 제 생각에, 더 시각적인 접근은 항상 더 낫습니다. 특히 여러분이 그것이 무엇을 하는지 이해하기 위해 원본을 하루 종일 반복해서 읽고 싶지 않다면요.

***

## Understanding Region of Interest - (RoI Pooling)
*아래 글은 [여기](https://towardsdatascience.com/understanding-region-of-interest-part-1-roi-pooling-e4f5dd65bb44)글을 해석한 글입니다.*  

![RoIPooling](https://miro.medium.com/max/700/0*lbtiFIOBUetVXsZt.png)  

우리는 여기서 Fast R-CNN 논문에 있는 오리지널 RoI Pooling에 대해 설명할 것이다. (위 그림에서 하늘색 네모부분) 그 과정에 두 번째와 세 번째 버전은 RoIAlign과 RoIWarp이다.  

### RoI는 무엇인가?
- RoI(Region of Interest)는 원본 이미지로부터 나온 제안된 영역이다. 여기서는 그 영역들을 추출하는 방법을 설명하지 않을 것이다. 왜냐하면 그것을 위한 여러 가지 방법들이 있기 때문이다. 지금 우리가 알아야 할 것은 바로 그런 영역이 여러 군데 있다는 것이고 그 영역들은 모두 마지막에 테스트를 해봐야 한다는 것이다.  

### Fast R-CNN은 어떻게 동작하는가?
**특성 추출**  
- `Fast R-CNN`은 기본 `R-CNN`네트워크와 다르다. 이것은 오직 하나의 컨볼루션 특성 추출을 가지고 있다. (여기에서는 VGG16을 예시로 사용할 것이다.)  

![VGG16](https://miro.medium.com/max/700/0*OVMjbYdDqpf3avlh.png)  

우리의 모델은 **512x512x3 (width x height x RGB)** 크기의 입력 이미지를 받고 VGG16을 통해 **16x15x512**로 특성 맵이 매핑된다. 당신은 다른 크기의 입력 이미지를 사용할 수 있다. (keras에서의 VGG16는 224x224 크기의 입력을 받는다.)  
만약 당신이 출력 매트릭스를 봤으면 이것의 **너비**와 **높이**가 정확히 입력 크기의 32배 차이나는 것을 알 수 있다. (512/32=16) 이것은 중요하다. 왜냐하면 모든 RoI들은 이 요소(32)에 의해 scale down이 되기 때문이다.  

### 샘플 RoIs
여기에 4개의 서로 다른 RoI들이 있다. 실제로 Fast R-CNN에서는 수천 개의 이미지가 있을 수 있지만 모두 인쇄하면 이미지를 읽을 수 없게 된다.  

![4RoIs](https://miro.medium.com/max/700/0*m3fiNgEsOg1yczlc.jpg)  

**RoI가 바운딩 박스가 아닌 점**을 기억하고 있어야 한다. 이것은 마치 그렇게 보일 수도 있지만 이것은 단지 이후의 process의 하나의 제안일 뿐이다. 많은 사람들은 대부분의 논문과 블로그 게시물들이 실제 객체 대신에 제안(proposals)을 만들고 있기 때문에 그렇게 추측하고 있다. 그 방법이 좀 더 편하기에 여기서도 이러한 방식을 택했다. *(편집자 주: 많은 사람들이 RoI를 설명할 때 실제 객체를 대표할 수 있는 part로 소개하고 있다는 의미로 생각됨)* 다음은 Fast R-CNN(녹색 상자)에 의해 확인될 다른 제안 영역의 예시이다.  

![proposals](https://miro.medium.com/max/700/0*6_xL9QmcBj0x2tZ7.jpg)  

RoI의 수를 제한하는 방법들이 있고 그것들에 대해 아래에 기술할 것이다.  

### 어떻게 특성 맵으로부터 RoI들을 얻을 수 있을까?
이제 RoI가 어떤 것인지 알았으니 VGG16의 출력 특성 맵에 매핑할 수 있어야 한다.  

![mapping](https://miro.medium.com/max/700/0*Hu43KfpLLgJ3C8SP.png)  

모든 RoI는 그것의 원본 좌표값과 크기를 가지고 있다. 이제 우리는 아래 하나에만 집중해보고자 한다.  
![focus](https://miro.medium.com/max/700/0*Ye1ZZL6zn6QXWbxw.jpg)  

이것의 원본 크기는 **145x200**이다. 그리고 좌측 상단의 좌표는 **(192, 296)** 이다. 알다시피 우리는 대부분의 숫자를 scale factor(32)로 나눴다.  

- width: 200/32 = 6.25  
- height: 145/32 = ~4.53  
- x: 296/32 = 9.25  
- y: 192/32 = 6  

오직 마지막 요소인 y만이 이치에 맞다. *(편집자 주: 계산 결과가 정수이다.)* 그 이유는 현재 16x16 그리드에서 작업하고 있기 때문이며, 우리가 관심을 가지는 형태는 정수이기 때문.(정확히는 자연수)  

### Quantization of coordinates on the feature map
**특성 맵의 좌표 양자화(정량화)**  
*편집자 주: Quantization의 정확한 뜻은 모르겠으나 특성 맵 좌표를 잘게 보는 관점인 듯. 퀀터제이션에서 퀀터는 물리학에서 말하는 양자역학 중 '양자'를 뜻하기 때문*  

Quantization(양자화)는 큰 값의 집합(실수와 같은)에서 이산 집합(정수와 같은)으로 입력을 제한하는 과정이다.  
*편집자 주: 실수와 같은 큰 범위의 입력이 아닌 좀 더 작은 범위의 정수 형태로 입력을 제한한다는 의미*  

만약 우리가 입력한 원본 특성 맵의 RoI가 아래와 같다면:  
![Original RoI on the feature map](https://miro.medium.com/max/700/0*itjjLvxulVdeHboj.png)  

일부 'cell'들이 분할되어 있기 때문에 풀링 레이어를 적용할 수 없다. 퀀터제이션은 행렬에 입력하기 전에 모든 결과를 내림(round down)을 적용한다. 9.25는 9가 되고 4.53은 4가 된다.  

![Quantized RoI](https://miro.medium.com/max/700/0*8t04t1gf-7UiaQoi.png)  

당신은 (아래 그림을 통해) 많은 양의 데이터를 잃은 것을 확인할 수 있다.(진한 파랑색) 그리고 새로운 데이터를 얻은 것을 알 수 있다.(초록색)  

![Quantization losses](https://miro.medium.com/max/700/0*W_0kPRvx-vkLxMNm.png)  

우리는 잃고 얻은 데이터에 대해 처리할 필요는 없다. 왜냐하면 아직 진행 중이기 때문에. 하지만 RoIAlign이라는 다른 버전의 프로세스가 이를 고칠 수 있다.  

### RoI Pooling
이제 RoI를 특성 맵에 매핑하면 풀링을 적용할 수 있다. 다시 한 번 RoI 풀링 레이어의 크기를 선택하겠다. 하지만 크기는 다를 수 있다. "RoI 풀링을 적용하는 이유가 무엇입니까?"라고 물을 수도 있습니다. 좋은 질문입니다. Fast R-CNN의 원래 디자인을 살펴보면 다음과 같습니다.  

![Original Fast R-CNN architecture](https://miro.medium.com/max/700/0*IoA3OWCLan-JDicO.png)  

**RoI 풀링 레이어**를 지난 후 고정된 크기의 Fully Connected 레이어가 있다. 왜냐하면 우리의 RoI들은 다른 크기를 가지고 있어서 같은 크기로 풀링해야 한다. (예: 3x3x512) 현재 우리의 지도화된 RoI는 4x6x512의 크기이며, 당신이 상상할 수 있듯이 우리는 4를 3으로 나눌 수 없다. 바로 여기서 양자화(quantization)가 다시 일어난다.  

![Mapped RoI and pooling layer](https://miro.medium.com/max/700/0*hWRZgO7YQTqjquIA.png)  

이번에는 좌표를 다룰 필요가 없고 크기만 다루면 된다. **6**을 **3**으로 나눌 수 있고 **2**를 얻을 수 있는 것은 행운이지만 **4**를 **3**으로 나누면 **1.33**이 남는다. 동일한 방법을 적용한 후(**내림**) 우리는 **1x2 벡터**를 갖게 된다. 매핑은 다음과 같다.  

![Data Pooling mapping](https://miro.medium.com/max/700/0*_6sxhtyFjdjVSi2q.png)  

양자화(quantization)때문에 우리는 최하단 row를 잃게 되었다.  

![Data Pooling maaping](https://miro.medium.com/max/621/0*lGS0NBC7tqpIxs6O.png)  

이제 우리는 **3x3x512**행렬에 데이터를 pool할 수 있다.  
![Data pooling process](https://miro.medium.com/max/700/0*SMrADTLV_u9SQ6EV.gif)  

이 경우 Max pooling을 적용했지만 모델에 따라 달라질 수 있다. 물론 이 과정은 최상위 레이어뿐만 아니라 전체 RoI행렬에서도 이루어진다. 최종결과는 다음과 같다.  

![Full-size pooling output](https://miro.medium.com/max/528/0*PV3fmXrcohjrGFo_.png)  

동일한 프로세스가 원본 이미지의 모든 RoI에 적용되므로 결국 수백 또는 수천 개의 3x3x512 매트릭스를 가질 수 있다. 이러한 매트릭스는 모두 나머지 네트워크(FC 계층에서 시작)를 통해 전송되어야 한다. 각 모델에 대해 별도로 bbox와 클래스를 생성하게 된다.  

## What next?
풀링이 완료된 후에는 입력 크기가 3x3x512이므로 FC 계층으로 전송하여 추가 처리를 수행할 수 있다. 한 가지 더 의논할 것이 있다. 정량화 과정으로 인해 많은 데이터가 손실되다. 정확히 말하자면, 그 정도입니다.  

![Data lost in quantization (dark and light blue), data gain(green)](https://miro.medium.com/max/700/0*R7FSiz6sFhU7QNBb.png)  

이는 각 셀이 엄청난 양의 데이터를 포함되어 있기 때문에 문제가 될 수 있다. 원본 이미지로부터 32x32x3의 이미지가 특성 맵으로 1x1x512로 변환되었는데 이것은 컨볼루션 레이어가 동작하는 방식이 아니므로 참조는 사용하지 말아달라. 그 문제를 해결할 방법(RoI Align)이 있는데 곧 두 번째 article를 작성할 예정이다.  

***

# FCN
Fully ConVolution Network  

1. FCN은 AlexNet, VGG-16(이미지넷 챌린지에서 좋은 성적 거둠) 등의 모델을 세그멘테이션에 맞게 변형.  
2. 기본적인 VGG 모델은 이미지의 특성을 추출하기 위해 **네트워크 뒷단에 Fully connected layer**를 붙여서 계산한 클래스별 확률을 바탕으로 이미지 분류를 수행.  
3. FCN에서는 세그멘테이션을 하기 위해 **네트워크 뒷단에 CNN**을 붙여줌.  

![F1](https://aiffelstaticprd.blob.core.windows.net/media/images/fcn.max-800x600.png)  
![F2](https://aiffelstaticprd.blob.core.windows.net/media/images/fcn_2.max-800x600.png)  

4. CNN을 붙여주는 이유: 이미지 내 위치의 특성을 유지하는 CNN을 이용해 **위치정보를 유지하면서 클래스 단위의 히트맵을 얻어 세그멘테이션을 하기 위해.**  
5. 마지막 CNN의 세팅: 위치의 특성을 유지하면서 이미지 분류를 하기 위해 **1x1의 커널 크기와 클래스의 개수만큼의 채널** 가짐.  
6. 히트맵의 크기는 원본보다 작다: 이유는 CNN과 pooling 레이어를 거치면서 크기가 줄어서. 키워주는 방법은 **upsampling**.  
7. Upsampling: FCN에서는 Deconvolution과 Interpolation 방식 활용.  
8. **Skip Architecture**: 아래 그림과 같이 FCN-32s, FCN-16s, FCN-8s의 결과를 볼 수 있는데, FCN-16s는 앞쪽 블록에서 얻은 예측 결과 맵과 2배로 upsampling한 맵을 더한 후, 한 번에 16배로 upsampling해주어 얻습니다. 여기서 한 번 더 앞쪽 블록을 사용하면 FCN-8s를 얻을 수 있다.  

![f3](https://aiffelstaticprd.blob.core.windows.net/media/images/fcn_3.max-800x600.png)  
![f4](https://aiffelstaticprd.blob.core.windows.net/media/images/fcn_4.max-800x600.png)

***

## U-Net
![u-net](https://aiffelstaticprd.blob.core.windows.net/media/images/u-net.max-800x600.png)  

1. 네트워크 구조가 U자 형태를 띠고 있다.  
2. **FCN에서 upsampling을 통해서 특성 맵을 키운 것을 입력값과 대칭적으로 만들어준 것**  
3. 본래 의학 관련 논문으로 시작되었음.  
4. 좌측 구조를 Contracting path, 우측 구조를 Expansive path.  
5. 좌측에서는 3x3 Convolution 계층과 Relu를 가지고 그 뒤로 downsampling을 위해서 2x2의 커널을 2stride로 max pooling을 하게 된다. downsampling을 거쳐 다음 convolution의 채널 크기는 두배씩 늘어나도록 설계됨.  
6. 우측에서는 각 블록에 2x2 up-convolution이 붙어 채널이 절반씩 줄어들고 특성 맵의 크기는 늘어난다. 좌측과 마찬가지로 3x3 convolution이 두 개씩 적용됨.  
7. 두 path에서 크기가 같은 블록의 출력과 입력은 **skip connection처럼 연결해주어 low-level의 feature를 활용**할 수 있도록 했다. 마지막에는 1x1 convolution으로 원하는 시맨틴 세그멘테이션 맵을 얻을 수 있다.  
8. 결과적으로 입력으로 572x572 크기인 이미지가 들어가고, 출력으로 388x388의 크기에 두 가지의 클래스를 가진 세그멘테이션 맵이 나온다.  


**Q5. U-Net의 구조는 대칭이지만 (원 논문의 구현을 기준으로 하면) 모델의 입력값과 출력값의 크기가 다릅니다. 그 이유가 뭘까요?**  
Convolution은 padding을 통해서 크기를 같게 유지할 수 있으나, U-Net에선 padding을 하지않아서 deconvolution으로 확대하더라도 원래 이미지 크기가 될 수 없습니다.  

### 타일(Tile) 기법
![tile](https://aiffelstaticprd.blob.core.windows.net/media/images/unet.max-800x600.png)  
- FCN은 입력 이미지의 크기를 조정하여 세그멘테이션 맵을 얻어냈는데, U-Net은 타일 방식을 사용해서 어느 정도 서로 겹치는 구간으로 타일을 나눠 네트워크를 추론, 큰 이미지에서도 높은 해상도의 세그멘테이션 맵을 얻을 수 있도록 했다.  

### 데이터 불균형 해결
1. 세포를 검출하기 위해서는 세포의 경계 예측이 중요.  
2. 픽셀 단위로 라벨링을 한다면, 데이터 셋에서 세포 간 경계의 면적이 작을 것.  
3. **클래스 간 데이터 양의 불균형을 해결**해주기 위해서 분포를 고려한 **weight map**을 학습 때 사용.  
4. 여기서 weight map의 weight는 **손실 함수(loss)에 적용되는 가중치**를 뜻한다.  
5. 의료 영상에서 세포 내부나 배경보다는 상대적으로 면적이 작은 세포 경계를 명확하게 추론해 내는 것이 더욱 중요하기 때문에, 세포 경계의 손실에 더 많은 패널티를 부과하는 방식

***

## DeepLab 계열
- 많은 개선을 거쳐 DeepLabv3+ 나옴.  
- 개선 내용 중 Atrous Convolution(Dilated Convolution이라고도 불림)와 Spatial Pyramid Pooling 등.  

### 전체 구조
![DeepLab](https://aiffelstaticprd.blob.core.windows.net/media/images/deeplab_v3.max-800x600.png)  

1. 인코더: 이미지에서 필요한 정보를 특성으로 추철해내는 모듈  
2. 디코더: 추출된 특성을 이용해 원하는 정보를 예측하는 모듈  
3. DeepLabV3+는 Atrous Convolution을 사용(U-Net에서는 3x3 Convolution 사용)  
4. Atrous Convolution을 여러 크기에 다양하게 적용한 것이 ASPP(Astrous Spatial Pyramid Pooling)  
5. eepLab V3+는 ASPP가 있는 블록을 통해 특성을 추출하고 디코더에서 Upsampling을 통해 세그멘테이션 마스크를 얻고 있다.  

### Atrous Convoltion
![ac](https://aiffelstaticprd.blob.core.windows.net/media/original_images/atrous_conv_2.gif)  

- "띄엄띄엄 보는 컨볼루션"  
- 더 넓은 영역을 보도록 해주기 위한 방법으로 커널이 일정 간격으로 떨어져 있어 컨볼루션 레이어를 너무 깊게 쌓지 않아도 넓은 영역의 정보를 커버할 수 있다.  

**Dilated Convolution (확장된 Convolution)**  
![DC](https://cdn-images-1.medium.com/max/1200/1*SVkgHoFoiMZkjy54zM_SUw.gif)  
2D convolution using a 3 kernel with a dilation rate of 2 and no padding  
 
- dilation rate는 커널 사이의 간격을 정의. 위 그림은 dilation rate가 2인 3x3 커널을 이용. 이는 5x5 커널과 동일한 시야(view)를 가짐.  
- 넓은 시야가 필요하고 여러 Convoltion이나 큰 커널을 사용할 여유가 없는 경우 사용.  
- 공간적 특징을 유지하기 때문에 Segmentation에서 많이 사용.  

> 출처: [딥러닝에서 사용되는 여러 유형의 Convolution 소개](https://zzsza.github.io/data/2018/02/23/introduction-convolution/)  

### Spatial Pyramid Pooling
![SPP](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-5-L-SPP.max-800x600.png)  

1. 여러 가지 스케일로 `convolution`과 `pooling`을 하고 나온 다양한 특성을 연결(concatenate)을 해줌.  
2. 이를 통해 멀티스케일로 특성을 추축하는 것을 병렬로 수행하는 효과를 얻을 수 있다.  
3. 여기서 convolution을 **Atrous Convolution**으로 바꾸어 적용한 것을 Atrous Spatial Pyramid Pooling이라고 한다.  
4. 장점: **입력 이미지 크기와 관계없이 동일한 구조를 활용 가능. 제각기 다양한 크기와 비율을 가진 RoI영역에 대해 적용하기 유리**  

> 출처: [갈아먹는 Object Detection - Spatial Pyramid Pooling Network](https://yeomko.tistory.com/14)  


***

## Upsampling의 다양한 방법
### 1) Nearest Neighbor
![NN](https://aiffelstaticprd.blob.core.windows.net/media/original_images/upsampling1.png)  

- Nearest upsampling은 이름 그대로 scale을 키운 위치에서 원본에서 가장 가까운 값을 그대로 적용하는 방법  
- 2x2 matrix가 있을때 이를 2배로 키우면 4x4의 matrix 된다.  

### 2) Bilinear Interpolation
![bi](https://aiffelstaticprd.blob.core.windows.net/media/original_images/bi_interpolation.png)  

- 선형 보간법을 이용해 필요한 값을 메우는 방식(위에서 봤듯이 2x2 행렬을 4x4로 upsampling을 할 때 빈 값을 채워야 한다)  
- R1은 Q11, Q21의 x축 방향의 interpolation 결과  
- R2는 Q12, Q22의 x축 방향의 interpolation 결과  
- P는 R1, R2의 y축 방향의 interpolation 결과  

> 출처: [bskyvision의 선형보간법(linear interpolation)과 삼차보간법(cubic interpolation), 제대로 이해하자](https://bskyvision.com/m/789)  

### 3) Transposed Convolution
![TC](https://aiffelstaticprd.blob.core.windows.net/media/images/transposed_conv.max-800x600.jpg)  

- 학습할 수 있는 파라미터를 가진 Upsampling 방법  
- Convolution 연산을 거꾸로 한다. (원래의 Convolution 연산: 커널의 크기를 정의하고 입력된 Feature를 Window에 따라서 Output을 계산)  

> 출처: [zzsza님의 Up-sampling with Transposed Convolution 번역](https://zzsza.github.io/data/2018/06/25/upsampling-with-transposed-convolution/)